In [9]:
import os
import torch
import scipy.io
import argparse
import pandas as pd
import numpy as np

In [10]:
def compute_mAP(index, good_index, junk_index):
    ap = 0
    cmc = torch.IntTensor(len(index)).zero_()
    if good_index.size == 0:
        cmc[0] = -1
        return ap, cmc

    # remove junk_index
    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]

    # find good_index index
    ngood = len(good_index)
    mask = np.in1d(index, good_index)
    rows_good = np.argwhere(mask == True)
    rows_good = rows_good.flatten()

    cmc[rows_good[0]:] = 1
    for i in range(ngood):
        d_recall = 1.0 / ngood
        precision = (i + 1) * 1.0 / (rows_good[i] + 1)
        if rows_good[i] != 0:
            old_precision = i * 1.0 / rows_good[i]
        else:
            old_precision = 1.0
        ap = ap + d_recall * (old_precision + precision) / 2

    return ap, cmc[:20]

In [11]:
def evaluate(score, ql, qc, gl, gc):
    # predict index sort from small to large
    index = np.argsort(score)[::-1]
    # good index
    query_index = np.argwhere(gl == ql)
    camera_index = np.argwhere(gc == qc)
    good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    #print(good_index)
    junk_index1 = np.argwhere(gl == -1)
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1)
    #print(junk_index)
    top_labels = []
    actual_labels = []
    for val in index[:10]:
        top_labels.append(gl[val])
    for val in good_index:
        actual_labels.append(gl[val])
    #print(index[:10], top_labels)
    return compute_mAP(index, good_index, junk_index)


In [13]:
torch.cuda.set_device(0)
logPath = '../../data/Market/extracted_dense_attr_64_color'
labelPath = '../../data/Market/extracted_dense_attr_64_color'
logPath2 = '../../data/Market/extracted_dense_attr_64_color_erasing'
labelPath2 = '../../data/Market/extracted_dense_attr_64_color_erasing'
logPath3 = '../../data/Market/extracted_3'
labelPath3 = '../../data/Market/extracted_3'

logFile = {subset: scipy.io.loadmat(os.path.join(logPath, 'feature_val_%s.mat' % subset))
           for subset in ['query', 'gallery']}
labelDict = {subset: pd.read_csv(os.path.join(labelPath, subset + 'Info.txt'), header=None,
                                 delimiter='\t').set_index(0)[1].to_dict() for subset in ['query', 'gallery']}

logFile2 = {subset: scipy.io.loadmat(os.path.join(logPath2, 'feature_val_%s.mat' % subset))
           for subset in ['query', 'gallery']}
labelDict2 = {subset: pd.read_csv(os.path.join(labelPath2, subset + 'Info.txt'), header=None,
                                 delimiter='\t').set_index(0)[1].to_dict() for subset in ['query', 'gallery']}
logFile3 = {subset: scipy.io.loadmat(os.path.join(logPath3, 'feature_val_%s.mat' % subset))
           for subset in ['query', 'gallery']}
labelDict3 = {subset: pd.read_csv(os.path.join(labelPath3, subset + 'Info.txt'), header=None,
                                 delimiter='\t').set_index(0)[1].to_dict() for subset in ['query', 'gallery']}

names = {subset: logFile[subset]['names'] for subset in ['query', 'gallery']}
labels = {subset: np.array([labelDict[subset][name] for name in names[subset]]) for subset in ['query', 'gallery']}
features = {subset: torch.FloatTensor(logFile[subset]['features']).cuda() for subset in ['query', 'gallery']}

names2 = {subset: logFile2[subset]['names'] for subset in ['query', 'gallery']}
labels2 = {subset: np.array([labelDict2[subset][name] for name in names[subset]]) for subset in ['query', 'gallery']}
features2 = {subset: torch.FloatTensor(logFile2[subset]['features']).cuda() for subset in ['query', 'gallery']}

names3 = {subset: logFile3[subset]['names'] for subset in ['query', 'gallery']}
labels3 = {subset: np.array([labelDict3[subset][name] for name in names[subset]]) for subset in ['query', 'gallery']}
features3 = {subset: torch.FloatTensor(logFile3[subset]['features']).cuda() for subset in ['query', 'gallery']}

In [14]:
CMC = torch.IntTensor(20).zero_()
ap = 0.0
ap_map_values = dict()
for i in range(len(labels['query'])):
    score = torch.mm(features['gallery'], (features['query'][i]).view(-1, 1))
    score = score.squeeze(1).cpu().numpy()
    ap_tmp, CMC_tmp = evaluate(score, labels['query'][i], [], labels['gallery'], [])
    ap_map_values[labels['query'][i]] = ap_tmp
    if CMC_tmp[0] == -1:
            continue
    CMC += CMC_tmp
    ap += ap_tmp
CMC = CMC.float()
CMC /= len(labels['query'])
ap /= len(labels['query'])
print('top1: %.4f, top5: %.4f, top10: %.4f, mAP: %.4f' % (CMC[0], CMC[4], CMC[9], ap))

top1: 0.9267, top5: 0.9733, top10: 0.9933, mAP: 0.8286


In [15]:
CMC = torch.IntTensor(20).zero_()
ap = 0.0
ap_map_values = dict()
for i in range(len(labels['query'])):
    score = torch.mm(features2['gallery'], (features2['query'][i]).view(-1, 1))
    score = score.squeeze(1).cpu().numpy()
    ap_tmp, CMC_tmp = evaluate(score, labels2['query'][i], [], labels2['gallery'], [])
    ap_map_values[labels2['query'][i]] = ap_tmp
    if CMC_tmp[0] == -1:
            continue
    CMC += CMC_tmp
    ap += ap_tmp
CMC = CMC.float()
CMC /= len(labels2['query'])
ap /= len(labels2['query'])
print('top1: %.4f, top5: %.4f, top10: %.4f, mAP: %.4f' % (CMC[0], CMC[4], CMC[9], ap))

top1: 0.9467, top5: 0.9667, top10: 0.9800, mAP: 0.8352


In [16]:
CMC = torch.IntTensor(20).zero_()
ap = 0.0
ap_map_values = dict()
for i in range(len(labels3['query'])):
    score = torch.mm(features3['gallery'], (features3['query'][i]).view(-1, 1))
    score = score.squeeze(1).cpu().numpy()
    ap_tmp, CMC_tmp = evaluate(score, labels3['query'][i], [], labels3['gallery'], [])
    ap_map_values[labels3['query'][i]] = ap_tmp
    if CMC_tmp[0] == -1:
            continue
    CMC += CMC_tmp
    ap += ap_tmp
CMC = CMC.float()
CMC /= len(labels3['query'])
ap /= len(labels3['query'])
print('top1: %.4f, top5: %.4f, top10: %.4f, mAP: %.4f' % (CMC[0], CMC[4], CMC[9], ap))

top1: 0.8733, top5: 0.9467, top10: 0.9600, mAP: 0.7757


In [17]:
CMC = torch.IntTensor(20).zero_()
ap = 0.0
ap_map_values = dict()
for i in range(len(labels['query'])):
    gallery_tensor =  features2['gallery'] + features['gallery']
    query_tensor =   features2['query'][i] + features['query'][i]
    score = torch.mm(gallery_tensor, query_tensor.view(-1, 1))
    score = score.squeeze(1).cpu().numpy()
    ap_tmp, CMC_tmp = evaluate(score, labels2['query'][i], [], labels2['gallery'], [])
    ap_map_values[labels2['query'][i]] = ap_tmp
    if CMC_tmp[0] == -1:
            continue
    CMC += CMC_tmp
    ap += ap_tmp
CMC = CMC.float()
CMC /= len(labels2['query'])
ap /= len(labels2['query'])
print('top1: %.4f, top5: %.4f, top10: %.4f, mAP: %.4f' % (CMC[0], CMC[4], CMC[9], ap))

top1: 0.9133, top5: 0.9733, top10: 0.9933, mAP: 0.8349
